In [3]:
from google import genai
import json

client = genai.Client()

In [5]:
json_file_path = 'batch_requests_with_pdfs.json'

print(f"Uploading JSONL file: {json_file_path}")
batch_input_file = client.files.upload(
    file=json_file_path
    )
print(f"Uploaded JSONL file: {batch_input_file.name}")

print("\nCreating batch job...")
batch_job_from_file = client.batches.create(
    model="gemini-2.5-flash",
    src=batch_input_file.name,
    config={
        'display_name': 'extraction_attempt_3',
    }
)
print(f"Created batch job from file: {batch_job_from_file.name}")
print("You can now monitor the job status using its name.")

Uploading JSONL file: batch_requests_with_pdfs.json
Uploaded JSONL file: files/cpd9y9uly1l8

Creating batch job...
Created batch job from file: batches/ff0deax8tnaay5rr0gqkn6g22pl3pd4orcrk
You can now monitor the job status using its name.


In [7]:
completed_states = set([
    'JOB_STATE_SUCCEEDED',
    'JOB_STATE_FAILED',
    'JOB_STATE_CANCELLED',
    'JOB_STATE_EXPIRED',
])

print("Listing recent batch jobs:\n")

# Note: The list API currently doesn't return inlined_responses.
# As a workaround,you can make a `get` call for inline jobs to see their results.
batches = client.batches.list(config={'page_size': 10})

for b in batches.page:
    print(f"Job Name: {b.name}")
    print(f"  - Display Name: {b.display_name}")
    print(f"  - State: {b.state.name}")
    print(f"  - Create Time: {b.create_time.strftime('%Y-%m-%d %H:%M:%S')}")
    if b.state.name in completed_states:
        print(f"  - End Time: {b.end_time.strftime('%Y-%m-%d %H:%M:%S')}")




    # Check if it was an inline job (no destination file)
    if b.dest is not None:
      if not b.dest.file_name:
        full_job = client.batches.get(name=b.name)
        if full_job.inlined_responses:
            print("  - Type: Inline ({} responses)".format(len(full_job.inlined_responses)))
      else:
          print(f"  - Type: File-based (Output: {b.dest.file_name})")

    print("-" * 20)

Listing recent batch jobs:

Job Name: batches/ff0deax8tnaay5rr0gqkn6g22pl3pd4orcrk
  - Display Name: extraction_attempt_3
  - State: JOB_STATE_PENDING
  - Create Time: 2025-09-11 22:12:59
--------------------
Job Name: batches/vnb8p71ws0ug2mgdkb6vmo11xlxspy40okyd
  - Display Name: my-batch-job-with-file-and-thoughts-thinking_config
  - State: JOB_STATE_SUCCEEDED
  - Create Time: 2025-09-11 16:44:56
  - End Time: 2025-09-11 17:48:30
  - Type: File-based (Output: files/batch-vnb8p71ws0ug2mgdkb6vmo11xlxspy40okyd)
--------------------
Job Name: batches/f9twk9y9jhwrgfyjrpq9h3ucnqr5cypgorio
  - Display Name: my-batch-job-with-file-and-thoughts
  - State: JOB_STATE_SUCCEEDED
  - Create Time: 2025-09-11 00:14:25
  - End Time: 2025-09-11 03:29:17
  - Type: File-based (Output: files/batch-f9twk9y9jhwrgfyjrpq9h3ucnqr5cypgorio)
--------------------
Job Name: batches/zi9hlczk1uwzljrevytef2qwqw41djv4wfnq
  - Display Name: my-batch-job-with-file
  - State: JOB_STATE_SUCCEEDED
  - Create Time: 2025-09